In [7]:
#%pip install numpy pandas scikit-learn nltk tensorflow keras

In [8]:
import numpy as np
import pandas as pd
import re
import nltk
#import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [9]:
df = pd.read_csv("D:/IMDB/IMDB Dataset.csv")  # Make sure the dataset is in the correct path
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

In [11]:
nltk.download('stopwords')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]  # Remove stopwords
    return ' '.join(words)

df['cleaned_review'] = df['review'].apply(clean_text)

[nltk_data] Downloading package stopwords to c:\Users\Muhammed
[nltk_data]     Salman\anaconda3\envs\tf\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt_tab to c:\Users\Muhammed
[nltk_data]     Salman\anaconda3\envs\tf\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [12]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X = tfidf_vectorizer.fit_transform(df['cleaned_review']).toarray()
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8925
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      4961
           1       0.88      0.91      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [14]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['cleaned_review'])

X_seq = tokenizer.texts_to_sequences(df['cleaned_review'])
X_padded = pad_sequences(X_seq, maxlen=200)

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

model = Sequential([
    Embedding(10000, 128, input_length=200),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")

Epoch 1/5
1250/1250 [==============================] - 476s 380ms/step - loss: 0.3680 - accuracy: 0.8369 - val_loss: 0.2778 - val_accuracy: 0.8850
Epoch 2/5
1250/1250 [==============================] - 476s 381ms/step - loss: 0.2229 - accuracy: 0.9141 - val_loss: 0.2702 - val_accuracy: 0.8844
Epoch 3/5
1250/1250 [==============================] - 477s 382ms/step - loss: 0.1676 - accuracy: 0.9377 - val_loss: 0.3273 - val_accuracy: 0.8816
Epoch 4/5
1250/1250 [==============================] - 478s 382ms/step - loss: 0.1352 - accuracy: 0.9508 - val_loss: 0.3286 - val_accuracy: 0.8787
Epoch 5/5
313/313 [==============================] - 31s 100ms/step - loss: 0.3850 - accuracy: 0.8784
Test Accuracy: 87.84%


In [27]:
def predict_sentiment(review):
    review = clean_text(review)
    review_seq = tokenizer.texts_to_sequences([review])
    review_padded = pad_sequences(review_seq, maxlen=200)
    prediction = model.predict(review_padded)
    return "Positive" if prediction > 0.5 else "Negative"

print(predict_sentiment("The movie was shit!"))

1/1 [==============================] - 0s 75ms/step
Negative


In [28]:
model.save("sentiment_model.h5")

In [29]:
import pickle

with open("tokenizer.pkl", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
!pip show tensorflow

Name: tensorflow
Version: 2.10.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\muhammed salman\anaconda3\envs\tf\lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, keras-preprocessing, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: 


In [19]:
!pip show flask

Name: Flask
Version: 3.1.0
Summary: A simple framework for building complex web applications.
Home-page: 
Author: 
Author-email: 
License: 
Location: c:\users\muhammed salman\anaconda3\envs\tf\lib\site-packages
Requires: blinker, click, itsdangerous, Jinja2, Werkzeug
Required-by: 


   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ----------- ---------------------------- 2.4/8.0 MB 11.2 MB/s eta 0:00:01
   ------------------------ --------------- 5.0/8.0 MB 12.1 MB/s eta 0:00:01
   ------------------------------- -------- 6.3/8.0 MB 12.1 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.6 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------  2.6/2.6 MB 6.9 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 6.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   -------------------------------------- - 2.1/2.2 MB 9.8 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 8.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
